In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Example data
c = [1, 2]  # Objective coefficients for x
d = [3, 4]  # Objective coefficients for y
A = [[1, 1], [2, 1]]  # Constraint coefficients for x
B = [[1, 0], [0, 1]]  # Constraint coefficients for y
b = [1, 2]  # Right-hand side

# Master problem (binary variables y)
master = gp.Model("master")
y = master.addVars(len(d), vtype=GRB.BINARY, name="y")
theta = master.addVar(name="theta")  # Variable for the Benders cut

master.setObjective(gp.quicksum(d[i] * y[i] for i in range(len(d))) + theta, GRB.MINIMIZE)
master.update()

# Set time limit to 60 seconds
master.setParam(GRB.Param.TimeLimit, 60)

# Set iteration limit to 10000
master.setParam(GRB.Param.IterationLimit, 10000)

# Set node limit to 1000
master.setParam(GRB.Param.NodeLimit, 1000)

# Enable lazy constraints
master.setParam(GRB.Param.LazyConstraints, 1)

# Subproblem (continuous variables x, given fixed y)
def solve_subproblem(fixed_y):
    subproblem = gp.Model("subproblem")
    x = subproblem.addVars(len(c), lb=0, name="x")
    
    subproblem.setObjective(gp.quicksum(c[j] * x[j] for j in range(len(c))), GRB.MINIMIZE)
    
    subproblem.addConstrs((gp.quicksum(A[i][j] * x[j] for j in range(len(c))) >= b[i] - gp.quicksum(B[i][j] * fixed_y[j] for j in range(len(d)))
                           for i in range(len(b))), name="constraints")
    
    subproblem.optimize()
    
    if subproblem.status == GRB.OPTIMAL:
        return subproblem.objVal, subproblem.getAttr('Pi', subproblem.getConstrs())
    else:
        return None, None

# Callback function for Benders cuts
def benders_callback(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get the values of the binary variables
        fixed_y = model.cbGetSolution([model.getVarByName(f"y[{i}]") for i in range(len(d))])
        
        # Solve the subproblem with fixed y
        sub_objval, duals = solve_subproblem(fixed_y)
        
        if sub_objval is not None:
            # Get the value of theta in the current solution
            theta_val = model.cbGetSolution(model.getVarByName("theta"))
            
            # Add Benders cut if necessary
            if theta_val < sub_objval:
                expr = sub_objval
                for i in range(len(b)):
                    expr += duals[i] * (gp.quicksum(B[i][j] * model.getVarByName(f"y[{j}]") for j in range(len(d))) - b[i])
                
                model.cbLazy(model.getVarByName("theta") >= expr)

# Set the callback function
master.optimize(benders_callback)

# Extract the solution
if master.status == GRB.OPTIMAL:
    print("Optimal value:", master.objVal)
    print("Optimal y:", [y[i].x for i in range(len(d))])
    print("Optimal theta:", theta.x)
else:
    print("No optimal solution found within the given limits")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-19
Set parameter TimeLimit to value 60
Set parameter IterationLimit to value 1000
Set parameter NodeLimit to value 1000
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700KF, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0xa41905a7
Variable types: 1 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700KF, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimi

GurobiError: Must set LazyConstraints parameter to use lazy constraints

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\\gurobipy\\callback.pxi", line 208, in gurobipy.CallbackClass.callback
  File "C:\Users\Yuyang\AppData\Local\Temp\ipykernel_35836\2999105380.py", line 61, in benders_callback
  File "src\\gurobipy\\model.pxi", line 6996, in gurobipy.Model.cbLazy
  File "src\\gurobipy\\model.pxi", line 7042, in gurobipy.Model._cbCutOrLazy
gurobipy.GurobiError: Must set LazyConstraints parameter to use lazy constraints


Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.13 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 24, time in user-callback 0.12 sec
Optimal value: 0.0
Optimal y: [-0.0, -0.0]
Optimal theta: 0.0
